
# Lab: Detecção de Anomalias Offshore (SGD Online) + **Produtor MQTT**
### (atualizado: *event timestamp* baseado em `t_mid` + **produtor_unix_ts**)

Este notebook simula sinais estruturais de uma coluna de plataforma offshore, extrai features
em janelas deslizantes, treina **modelos rasos** via **SGD online** e publica mensagens **MQTT**.
Atualizações:

- Incluído **carimbo de tempo do evento** (`event_unix_ts`) derivado de `t_mid`.
- Incluído **carimbo de tempo do produtor** (`producer_unix_ts`) no momento do `publish`.
- Célula extra de **latência de detecção** (com base no dano simulado).
- Bloco de **instruções** para configurar o *bridge* (MQTT→InfluxDB) a usar `event_unix_ts` como `time`.

> Em produção, o *event timestamp* deve vir do domínio do sinal/medição; aqui usamos `t_mid` (centro da janela).


In [ ]:

# Requisitos (execute uma vez, se necessário)
# !pip install paho-mqtt --quiet
# !pip install torch --quiet


In [ ]:

import numpy as np
import pandas as pd
import math, json, time
from scipy.signal import welch, stft
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

try:
    import paho.mqtt.client as mqtt
    HAS_MQTT = True
except Exception as e:
    print("paho-mqtt não disponível:", e)
    HAS_MQTT = False

np.random.seed(42)
torch.manual_seed(42)

print("Versões -> numpy", np.__version__, "| pandas", pd.__version__, "| torch", torch.__version__)
print("MQTT disponível?", HAS_MQTT)


## Simulador (SDOF) com injeção de dano

In [ ]:

def simulate_sdof(T=1800, fs=200, m=1.0, c=0.02, k=1000.0, damage_t=900, delta=0.02,
                  sea_noise_level=0.1, accel_noise_level=0.02):
    n = int(T*fs)
    dt = 1.0/fs
    x = np.zeros(n); v = np.zeros(n); a = np.zeros(n)
    k_t = k
    forcing = sea_noise_level*np.random.randn(n) + 0.02*np.sin(np.linspace(0, 40*math.pi, n))
    strain = np.zeros(n)
    is_damaged = np.zeros(n, dtype=int)

    for i in range(1,n):
        if i == int(damage_t*fs):
            k_t = k*(1.0-delta)
        if i >= int(damage_t*fs):
            is_damaged[i] = 1
        a[i] = (forcing[i] - c*v[i-1] - k_t*x[i-1]) / m
        v[i] = v[i-1] + a[i]*dt
        x[i] = x[i-1] + v[i]*dt
        strain[i] = x[i] * k_t

    accel = a + accel_noise_level*np.random.randn(n)
    temp = 20 + 5*np.sin(np.linspace(0, 6*math.pi, n)) + 0.2*np.random.randn(n)
    wind = 5 + 2*np.sin(np.linspace(0, 10*math.pi, n)) + 0.5*np.random.randn(n)
    return accel, strain, temp, wind, fs, is_damaged


## Extração de features (f₁ via Welch, RMS, tendências)

In [ ]:

def estimate_f1(accel_win, fs, fmin=0.1, fmax=50.0):
    f, Pxx = welch(accel_win, fs=fs, nperseg=min(len(accel_win)//2, 512))
    mask = (f >= fmin) & (f <= fmax)
    if not np.any(mask):
        return np.nan
    f_slice = f[mask]
    P_slice = Pxx[mask]
    idx = np.argmax(P_slice)
    return float(f_slice[idx])

def extract_features(accel_win, strain_win, temp_win, wind_win, fs):
    f1 = estimate_f1(accel_win, fs)
    rms = float(np.sqrt(np.mean(accel_win**2)))
    x_idx = np.arange(len(strain_win), dtype=float)
    st_mean = float(np.mean(strain_win))
    st_slope = float(np.polyfit(x_idx, strain_win, 1)[0]) if len(strain_win) > 1 else 0.0
    t_mean = float(np.mean(temp_win))
    w_mean = float(np.mean(wind_win))
    return np.array([f1, rms, st_mean, st_slope, t_mean, w_mean], dtype=np.float32)


## Modelos rasos (baseline linear + autoencoder) com **SGD online**

In [ ]:

class LinearBaseline(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.lin = nn.Linear(d, 1, bias=False)
        nn.init.zeros_(self.lin.weight)
    def forward(self, u):
        return self.lin(u).squeeze(-1)

class AE(nn.Module):
    def __init__(self, d_in=5, h1=16, z=8):
        super().__init__()
        self.enc = nn.Sequential(nn.Linear(d_in, h1), nn.ReLU(), nn.Linear(h1, z))
        self.dec = nn.Sequential(nn.Linear(z, h1), nn.ReLU(), nn.Linear(h1, d_in))
    def forward(self, x):
        z = self.enc(x)
        return self.dec(z)


## Configuração MQTT + **timestamps**

In [ ]:

# Configuração MQTT
MQTT_BROKER = "localhost"
MQTT_PORT = 1883
MQTT_KEEPALIVE = 60
MQTT_TOPIC_ANOM = "plataforma/anomalia"

mqtt_client = None
if 'mqtt' in globals() and HAS_MQTT:
    try:
        mqtt_client = mqtt.Client()
        mqtt_client.connect(MQTT_BROKER, MQTT_PORT, MQTT_KEEPALIVE)
        print(f"Conectado ao broker MQTT {MQTT_BROKER}:{MQTT_PORT}")
    except Exception as e:
        print("Falha ao conectar no broker MQTT:", e)
else:
    print("paho-mqtt indisponível; a publicação MQTT será ignorada.")

# Base temporal para converter t_mid (s) em epoch
BASE_EVENT_UNIX_TS = time.time()
print("BASE_EVENT_UNIX_TS =", BASE_EVENT_UNIX_TS)


## Loop online: janelas, SGD, **publicação MQTT** com `event_unix_ts` e `producer_unix_ts`

In [ ]:

# Parâmetros
T = 1800
fs = 200
win_sec = 4.0
overlap = 0.5
win = int(win_sec*fs)
step = int(win*(1.0-overlap))

accel, strain, temp, wind, fs, is_damaged = simulate_sdof(T=T, fs=fs, damage_t=900, delta=0.02)

# Modelos
d_u = 6  # 5 features + 1 bias
baseline = LinearBaseline(d=d_u)
opt_lin = optim.SGD(baseline.parameters(), lr=1e-3)
ae = AE(d_in=5, h1=16, z=8)
opt_ae = optim.SGD(ae.parameters(), lr=1e-3)

hist = []
scores_hist = []

def publish_mqtt(payload: dict):
    if mqtt_client is None:
        return
    try:
        mqtt_client.publish(MQTT_TOPIC_ANOM, json.dumps(payload))
    except Exception as e:
        print("Erro ao publicar MQTT:", e)

for start in range(0, len(accel)-win, step):
    sl = slice(start, start+win)
    accw, stw, tw, ww = accel[sl], strain[sl], temp[sl], wind[sl]
    feats = extract_features(accw, stw, tw, ww, fs)
    f1 = feats[0]
    if not np.isfinite(f1):
        continue

    feats_lin = feats[1:]  # [rms, st_mean, st_slope, t_mean, w_mean]
    u = np.r_[feats_lin, 1.0].astype(np.float32)
    u_t = torch.from_numpy(u).unsqueeze(0)
    y_t = torch.tensor([f1], dtype=torch.float32)

    # Baseline linear (SGD)
    opt_lin.zero_grad()
    y_hat = baseline(u_t)
    loss_lin = (y_hat - y_t).pow(2).mean()
    loss_lin.backward()
    opt_lin.step()

    # Autoencoder (SGD)
    x_in = torch.from_numpy(feats_lin).unsqueeze(0)
    opt_ae.zero_grad()
    x_rec = ae(x_in)
    loss_ae = ((x_rec - x_in)**2).mean()
    loss_ae.backward()
    opt_ae.step()

    with torch.no_grad():
        y_hat_detached = baseline(u_t).item()
        score_pred = abs(f1 - y_hat_detached)
        score_rec = float(loss_ae.item())
        score = score_pred + score_rec
        scores_hist.append(score)
        thr = float(np.quantile(scores_hist[-120:], 0.99)) if len(scores_hist) > 120 else float("inf")
        alert = bool(score > thr)

    t_mid = (start + win//2) / fs                               # segundos desde o início da simulação
    event_unix_ts = BASE_EVENT_UNIX_TS + t_mid                  # timestamp do evento (centro da janela)
    producer_unix_ts = time.time()                              # timestamp do produtor no publish

    damaged = int(np.any(is_damaged[sl]))

    row = dict(t=t_mid, event_unix_ts=event_unix_ts, producer_unix_ts=producer_unix_ts,
               f1_obs=float(f1), f1_pred=float(y_hat_detached), 
               loss_ae=float(loss_ae.item()), score=float(score), thr=float(thr),
               alert=alert, damaged=damaged)
    hist.append(row)

    publish_mqtt({
        "event_unix_ts": event_unix_ts,
        "producer_unix_ts": producer_unix_ts,
        "t_mid": t_mid,
        "f1_obs": float(f1),
        "f1_pred": float(y_hat_detached),
        "score": float(score),
        "threshold": float(thr),
        "alert": alert,
        "damaged": damaged
    })

df = pd.DataFrame(hist)
print("Janelas processadas:", len(df))
df.head()


## Visualizações (2D)

In [ ]:

plt.figure()
plt.plot(df['t'], df['f1_obs'], label='f1_obs')
plt.plot(df['t'], df['f1_pred'], label='f1_pred')
plt.xlabel('Tempo (s)')
plt.ylabel('f1 (Hz)')
plt.title('f1 observado vs previsto')
plt.legend()
plt.show()


In [ ]:

plt.figure()
plt.plot(df['t'], df['score'], label='score')
plt.plot(df['t'], df['thr'], label='threshold')
if df['damaged'].any():
    t_damage_start = df.loc[df['damaged']==1, 't'].min()
    plt.axvspan(t_damage_start, df['t'].max(), alpha=0.2)
plt.xlabel('Tempo (s)')
plt.ylabel('Score')
plt.title('Score e limiar adaptativo')
plt.legend()
plt.show()


In [ ]:

plt.figure()
plt.plot(df['t'], df['alert'].astype(int), drawstyle='steps-post')
plt.xlabel('Tempo (s)')
plt.ylabel('Alerta (0/1)')
plt.title('Alertas')
plt.show()


## Visualização 3D (espectrograma STFT)

In [ ]:

T_demo = 60
fs_demo = 200
def simulate_for_3d(T=60, fs=200, damage_t=30.0, delta=0.03):
    accel, _, _, _, _, _ = simulate_sdof(T=T, fs=fs, damage_t=damage_t, delta=delta)
    return accel

accel_demo = simulate_for_3d(T=T_demo, fs=fs_demo, damage_t=30, delta=0.03)
f, tt, Zxx = stft(accel_demo, fs=fs_demo, nperseg=256, noverlap=128)
S = np.abs(Zxx)
mask = f <= 40
f_plot = f[mask]
S_plot = S[mask, :]

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
T_mesh, F_mesh = np.meshgrid(tt, f_plot)
ax.plot_surface(T_mesh, F_mesh, S_plot, linewidth=0, antialiased=True)
ax.set_xlabel('Tempo (s)')
ax.set_ylabel('Frequência (Hz)')
ax.set_zlabel('|STFT|')
ax.set_title('Espectrograma 3D (queda de rigidez aos 30s)')
plt.show()


## Métricas de desempenho + **latência de detecção**

In [ ]:

det_delay = np.nan
first_alert_ts = np.nan
first_alert_event_ts = np.nan

if df['damaged'].any() and df['alert'].any():
    t_damage = df.loc[df['damaged']==1, 't'].min()
    after = df[df['t'] >= t_damage]
    fired = after[after['alert'] == True]
    if len(fired) > 0:
        first_alert_t = float(fired['t'].iloc[0])
        det_delay = first_alert_t - float(t_damage)
        first_row = fired.iloc[0]
        first_alert_ts = float(first_row['producer_unix_ts'])
        first_alert_event_ts = float(first_row['event_unix_ts'])

pre = df[df['damaged']==0]
fpr_per_hour = np.nan
if len(pre) > 0:
    time_pre = pre['t'].max() - pre['t'].min()
    alerts_pre = int(pre['alert'].sum())
    if time_pre > 0:
        fpr_per_hour = float(alerts_pre / (time_pre/3600.0))

print("Atraso de detecção (s) (evento→primeiro alerta):", det_delay)
print("Falsos positivos por hora (aprox):", fpr_per_hour)
print("first_alert_event_ts:", first_alert_event_ts, "| first_alert_producer_ts:", first_alert_ts)


## (Opcional) Exportar histórico para CSV

In [ ]:

df.to_csv("historico_anomalias.csv", index=False)
print("Arquivo salvo: historico_anomalias.csv")



## Instruções para o *bridge* MQTT→InfluxDB (usar `event_unix_ts`)
No seu *bridge* (Python/InfluxDB client), mapeie:
- `time` (timestamp da medição) = `event_unix_ts` (segundos POSIX)
- `field`/`tags`: mantenha `producer_unix_ts` como *campo* para calcular **latência de publicação** (`producer_unix_ts - event_unix_ts`).

Em cenários reais, se o produtor souber o instante preciso da janela (ex.: *hardware timestamp*), substitua `BASE_EVENT_UNIX_TS` pelo tempo absoluto de início da aquisição.
